In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

In [2]:
data = pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
## Preprocess the data
### Drop irrelevant columns
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
data.isnull().sum()

CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [5]:
## Encode categorical variables
label_encoder_gender=LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [6]:
## Onehot encode 'Geography
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo=OneHotEncoder()
geo_encoder=onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoder

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [7]:
onehot_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [8]:
df_OHE = pd.DataFrame(geo_encoder,columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
df_OHE.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [9]:
data = pd.concat([data,df_OHE],axis=1)
data.drop(['Geography'],axis=1,inplace=True)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [10]:
with open('label_encoder.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)
with open('OHE_encoder.pkl','wb') as file:
    pickle.dump(onehot_encoder_geo,file)

In [11]:
## DiVide the dataset into indepent and dependent features
X=data.drop('Exited',axis=1)
y=data['Exited']

## Split the data in training and tetsing sets
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

## Scale these features
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

In [12]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [13]:
# shape returns tuple carrying values of dimension, to access the coloums we used 1 index --> (row,coloums)
X_train.shape[1]

12

In [14]:
## Build Our ANN Model
## For 1st HL we have to pass no of nodes in input layer 
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),  # HL1 Connected with input layer
    Dense(32, activation='relu'),  # HL2
    Dense(1, activation='sigmoid')  # output layer
])
model.summary()

/Applications/CODE/ETEANNDL/eteannml_venv/lib/python3.11/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [15]:
import tensorflow
opt=tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss_fn=tensorflow.keras.losses.BinaryCrossentropy()

In [16]:
## compile the model
model.compile(optimizer=opt,loss=loss_fn,metrics=['accuracy'])

In [17]:
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard

## Set up the Tensorboard
log_directory="logs/fit/"
tensorflow_callback=TensorBoard(log_dir=log_directory,histogram_freq=1)

## Set up Early Stopping
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [18]:
### Train the model
history=model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8218 - loss: 0.4229 - val_accuracy: 0.8545 - val_loss: 0.3568
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 791us/step - accuracy: 0.8505 - loss: 0.3590 - val_accuracy: 0.8535 - val_loss: 0.3719
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 783us/step - accuracy: 0.8562 - loss: 0.3541 - val_accuracy: 0.8590 - val_loss: 0.3524
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 738us/step - accuracy: 0.8645 - loss: 0.3302 - val_accuracy: 0.8650 - val_loss: 0.3396
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 751us/step - accuracy: 0.8711 - loss: 0.3256 - val_accuracy: 0.8575 - val_loss: 0.3479
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 752us/step - accuracy: 0.8714 - loss: 0.3257 - val_accuracy: 0.8610 - val_loss: 0.3404
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 755us/step - accuracy: 0.8672 - loss: 0.3278 - val_accuracy: 0.8630 - val_loss: 0.3352
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 737us/step - accuracy: 0.8619 - loss: 0.3

In [19]:
model.save('ANN_model.keras')

In [20]:
## Load Tensorboard Extension
%load_ext tensorboard

In [21]:
%tensorboard --logdir logs/fit/

Reusing TensorBoard on port 6009 (pid 54775), started 17:19:15 ago. (Use '!kill 54775' to kill it.)